https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/basic/Tutorial/1.Importing_Your_Data.ipynb

In [ ]:
# STEP 1 - CONNECT
import boto3
import datetime
import pandas as pd

In [ ]:
REGION = 'eu-west-1'
BUCKET_URI = 's3://mleipchapter7/data/rossman/'
ALGORITHM_ARN = 'arn:aws:forecast:::algorithm/Prophet'
PREDICTOR_BASE_NAME = 'store_demand_prophet'#+datetime.datetime.now().strftime(format='%Y_%m_%d')
DATASET_GROUP_ARN = 'arn:aws:forecast:eu-west-1:508972911348:dataset-group/store_demand_group'
DATASET_FREQUENCY = 'D'
FORECAST_BASE_NAME = 'store_model_prophet'

class ForecastSession(object):
    
    def __init__(self):
        self.session = boto3.Session(region_name=REGION) 
        self.forecast = self.session.client(service_name='forecast') 
        self.forecastquery = self.session.client(service_name='forecastquery')
        
class Trainer(object):
    
    def __init__(self, forecast_session):
        self.forecast_session = forecast_session
        self.df_pred_metadata = self.get_df_pred_metadata()
        self.latest_predictor = self.get_latest_predictor()
        
    def get_df_pred_metadata(self):
        predictor_metadata = self.forecast_session.forecast.list_predictors()['Predictors']
        df_pred_metadata = pd.DataFrame.from_records(predictor_metadata)
        return df_pred_metadata
        
    def get_latest_predictor(self):
        latest_predictor = self.df_pred_metadata.sort_values(by='CreationTime', ascending=False).loc[0].to_dict()
        return latest_predictor
        
    def latest_predictor_in_tolerance(self, tolerance_days=2):
        train_time_elapsed_days = (
            datetime.datetime.now()-self.latest_predictor['CreationTime'].replace(tzinfo=None)
        ).days
        if train_time_elapsed_days < tolerance_days:
            return True
        else:
            return False
        
        
    def train_new_predictor(self):
        PREDICTOR_NAME = PREDICTOR_BASE_NAME + datetime.datetime.now().strftime(format='%Y_%m_%d_%H_%M')
        train_response = self.forecast_session.forecast.create_predictor(PredictorName=PREDICTOR_NAME,
                              AlgorithmArn=ALGORITHM_ARN,
                              ForecastHorizon=7,
                              PerformAutoML=False,
                              PerformHPO=False,
                              InputDataConfig= {"DatasetGroupArn": DATASET_GROUP_ARN},
                              FeaturizationConfig= {"ForecastFrequency": DATASET_FREQUENCY}
                             )
        return train_response
    
    def create_latest_forecast(self):
        FORECAST_NAME = FORECAST_BASE_NAME + datetime.datetime.now().strftime(format='%Y_%m_%d_%H_%M')
        create_forecast_response = self.forecast_session.forecast.create_forecast(
            ForecastName=FORECAST_NAME,
            PredictorArn=self.latest_predictor['PredictorArn'])
        return create_forecast_response
        
    
    
class Forecaster(object):
    
    def __init__(self, forecast_session):
        self.forecast_session = forecast_session
        self.df_forecast_metadata = self.get_df_forecast_metadata()
        self.latest_forecast = self.get_latest_forecast()
        
    def get_df_forecast_metadata(self):
        forecast_metadata = self.forecast_session.forecast.list_forecasts()['Forecasts']
        df_forecast_metadata = pd.DataFrame.from_records(forecast_metadata)
        return df_forecast_metadata
        
    def get_latest_forecast(self):
        latest_forecast = self.df_forecast_metadata.sort_values(by='CreationTime', ascending=False).loc[0].to_dict()
        return latest_forecast

    def get_forecast(self, store_id):
        return self.forecast_session.forecastquery.query_forecast(
            ForecastArn=self.latest_forecast['ForecastArn'],
            Filters={"item_id": str(store_id)}
        )
    

In [ ]:
forecast_session = ForecastSession()

In [ ]:
trainer = Trainer(forecast_session)

In [ ]:
trainer.latest_predictor_in_tolerance()

In [ ]:
trainer.train_new_predictor()

In [ ]:
forecaster = Forecaster(forecast_session)

In [ ]:
forecaster.get_forecast(1)

In [ ]:
forecast_session.forecast.list_forecasts()

In [ ]:
forecaster_metadata = forecast_session.forecast.list_forecasts()['Forecasts']
df_forecast_metadata = pd.DataFrame.from_records(forecaster_metadata)

In [ ]:
df_forecast_metadata.sort_values(by='CreationTime', ascending=False).loc[0].to_dict()

In [ ]:
# STEP 1 - CONNECT
import boto3
import datetime

REGION = 'eu-west-1'
BUCKET_URI = 's3://mleipchapter7/data/rossman/'


# Connect API session
session = boto3.Session(region_name=REGION) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')


In [ ]:
forecast.list_predictors()

In [ ]:
#STEP 2 - CREATE NEW PREDICTOR

ALGORITHM_ARN = 'arn:aws:forecast:::algorithm/Prophet'
PREDICTOR_NAME = 'store_demand_prophet'+datetime.datetime.now().strftime(format='%Y_%m_%d')
DATASET_GROUP_ARN = 'arn:aws:forecast:eu-west-1:508972911348:dataset-group/store_demand_group'
DATASET_FREQUENCY = 'D'

create_predictor_response = \
    forecast.create_predictor(PredictorName=PREDICTOR_NAME,
                              AlgorithmArn=ALGORITHM_ARN,
                              ForecastHorizon=7,
                              PerformAutoML=False,
                              PerformHPO=False,
                              InputDataConfig= {"DatasetGroupArn": DATASET_GROUP_ARN},
                              FeaturizationConfig= {"ForecastFrequency": DATASET_FREQUENCY}
                             )



In [ ]:
forecast.list_predictors()['Predictors']

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame.from_records(forecast.list_predictors()['Predictors'])

In [ ]:
(datetime.datetime.now()-df1['CreationTime'].max().replace(tzinfo=None)).days

In [ ]:
predictor_metadata = forecast.list_predictors()['Predictors']
df_pred_metadata = pd.DataFrame.from_records(predictor_metadata)

In [ ]:
latest_predictor = df_pred_metadata.sort_values(by='CreationTime', ascending=False).loc[0].to_dict()['CreationTime']

In [ ]:
latest_predictor

In [ ]:
forecast.list_forecasts()